In [1]:
from queries import*
from utilities import*
from scipy.spatial import Delaunay
from numpy import linalg
from sklearn import preprocessing
import copy

In [ ]:
Directory = os.getcwd()
Directory_path = Directory + '/data/input'
files = os.listdir(Directory_path)
plot_Directory = Directory + '/plots'

for file in files[0:1]:
    
    array = file.split('_')
    if len(array)==2:
        condition = array[0]
        PATIENT_ID = array[1]
    else:
        condition == ''
        
    
    if condition == 'DEEPMEL':
        
        
        print('Treating patient' + PATIENT_ID)

        
        INPUT_PATH = './data/input/DEEPMEL_' + PATIENT_ID + '/'
        OUTPUT_PATH = './data/output/DEEPMEL_' + PATIENT_ID +'/'
        PLOT_PATH ='./plots/DEEPMEL_' + PATIENT_ID +'/'
        INPUT_FILE = 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data.csv'
        TABLE_NAME_BQ = 'DEEPMEL_'+ PATIENT_ID +'_cell_seg_data'
        SHOW_PLOTS =False   
        HIGH_VOLUME = True
        MIN_ELEMENTS_PER_CLUSTER = 100
        ### The border values will extract only the border degined in the edge_neighberhood
        BORDER = True
        
        
        nodes = pd.read_csv(OUTPUT_PATH+'nodes_with_border.csv')
        size_node_df =len(nodes)
      
                    
        ### Setting cell_id as index

        nodes.set_index('cell_id', inplace=True)
        #nodes['phenotype'] = nodes['phenotype'].apply(lambda x: get_phenotype(x))
        nodes_stats = pd.DataFrame(nodes.phenotype.value_counts()/len(nodes)).reset_index()
        nodes_stats = pd.pivot_table(nodes_stats,columns='index').reset_index()
        nodes_stats.columns.name = None 
        nodes_stats = nodes_stats.rename(columns={'index':'name'})
        nodes_stats['name'] = PATIENT_ID
        
        
        print('Importing edges...')
        edges = pd.read_csv(OUTPUT_PATH + 'graph_edges.csv')
        print('DONE')
        cells_in_edges = get_cells_from_edges(edges)
        
        print('Counting edges per phenotype...')
        N_EDGES = len(edges)
        nodes_stats['nbr_edges'] = N_EDGES
        edge_phenotype_count = edges.groupby(['phenotype_1','phenotype_2']).agg('count')[['cell_id_1']].reset_index().rename(columns = {'cell_id_1':'edge_count'})
        edge_phenotype_count['edge_type'] = edge_phenotype_count.apply(lambda row : order_phenotypes(row.phenotype_1,row.phenotype_2), axis=1)
        edge_phenotype_count = edge_phenotype_count[['edge_count','edge_type']]
        edge_count = copy.deepcopy(edge_phenotype_count)
        edge_count['edge_count'] = edge_phenotype_count['edge_count'].apply(lambda x: x/N_EDGES)
        edge_phenotype_count_pivotted = pd.pivot_table(edge_count, values='edge_count',columns=['edge_type'], aggfunc=np.sum)
        edge_phenotype_count_pivotted.columns.name = None
        edge_phenotype_count_pivotted = edge_phenotype_count_pivotted.reset_index()
        edge_phenotype_count_pivotted = edge_phenotype_count_pivotted.fillna(0)
        edge_phenotype_count_pivotted = edge_phenotype_count_pivotted.drop(['index'], axis=1)
        nodes_stats = nodes_stats.join(edge_phenotype_count_pivotted)
        del edge_phenotype_count_pivotted

        print('DONE')
        print('Computing AUC...')
        Min_distance = np.min(edges.distance)
        Max_distance = np.max(edges.distance)
        distribution = G_cross_function(edges,Min_distance,Max_distance,1000)
        nodes_stats['AUC_total_graph'] = compute_AUC(distribution)
        print('DONE')

        print('Computing TILs...')

        TILs = ['T','B','NK']
        df_in_border_pivotted = get_infiltrated_tils(nodes, TILs, on_border=True)
        df_in_border_pivotted = df_in_border_pivotted.rename(columns = {'B':'B_in','NK':'NK_in','T':'T_in'})
        df_out_border_pivotted = get_infiltrated_tils(nodes, TILs, on_border=False)
        df_out_border_pivotted = df_out_border_pivotted.rename(columns = {'B':'B_out','NK':'NK_out','T':'T_out'})
        nodes_stats = nodes_stats.join(df_out_border_pivotted)
        nodes_stats = nodes_stats.join(df_in_border_pivotted)
        del df_out_border_pivotted
        del df_in_border_pivotted
        print('DONE')
        
        print('Cell degree...')
        
        cell_degree = pd.read_csv(OUTPUT_PATH+'cell_degree.csv')
        
        average_degree = pd.DataFrame(cell_degree.mean()).reset_index()
        average_degree = pd.pivot_table(average_degree,columns='index').reset_index()
        average_degree.columns.name =None 
        average_degree = average_degree.drop(['index','cell_id'], axis=1)
        #nodes_stats = nodes_stats.rename(columns={'index':'name'})
        nodes_stats = nodes_stats.join(average_degree)
        nodes = nodes.merge(cell_degree, on='cell_id',how='left')
        print('DONE')
        
        print('Building the whole graph...')
        whole_cell_graph =nx.from_pandas_edgelist(edges, 'cell_id_1', 'cell_id_2', edge_attr=None, create_using= nx.Graph())
        nodes_stats['whole_graph_sparsity'] = nx.density(whole_cell_graph)
        nodes_stats['whole_graph_transitivity'] = nx.transitivity(whole_cell_graph)
        del whole_cell_graph
        print('DONE')

        print('Building the border graph...')

        
        if BORDER:
            edges = edges[edges.is_border== True]
            nodes = nodes[nodes.on_border == True]
    
        cell_graph =nx.from_pandas_edgelist(edges, 'cell_id_1', 'cell_id_2', edge_attr=None, create_using= nx.Graph())
        describe_graph(cell_graph)
        nodes_stats['border_graph_sparsity'] = nx.density(cell_graph)
        nodes_stats['border_graph_transitivity'] = nx.transitivity(cell_graph)
        
        plot_degree_distribution(cell_graph,savefig = True, figname = 'Degreed_itribution_bipartite.png',PLOT_PATH=PLOT_PATH,no_show = SHOW_PLOTS );
        plt.close()

        nx.set_node_attributes(cell_graph, nodes['cell_x_position'].to_dict(), 'x_position' )
        nx.set_node_attributes(cell_graph, nodes['cell_y_position'].to_dict(), 'y_position' )
        nx.set_node_attributes(cell_graph, nodes['tissue_category'].to_dict(), 'tissue_category' )
        nx.set_node_attributes(cell_graph, nodes['phenotype'].to_dict(), 'phenotype' )
        nx.set_node_attributes(cell_graph, nodes['on_border'].to_dict(), 'border')
        print('DONE')
        
        
        print('Connected components...')

        components = list(nx.connected_components(cell_graph))
        nodes_stats['n_initial_components'] =  len(components)
        print('The cell graph contains', len(components), 'connected components')
        largest_comp = max(components, key=len)
        percentage_lcc = len(largest_comp)/cell_graph.number_of_nodes() * 100
        nodes_stats['size_largest_component'] =  len(components)

        print('The largest component has', len(largest_comp), 'nodes', 'accounting for %.2f'% percentage_lcc, '% of the nodes')
        components = filtering_components(components, MIN_ELEMENTS_PER_CLUSTER)
        components = sorted(components, key=len, reverse=False)
        print(len(components),' components have at least ',MIN_ELEMENTS_PER_CLUSTER,' nodes')
        nodes_stats['n_final_components'] =  len(components)
        
        
        clustering_coef = nx.clustering(cell_graph)
        clustering_df =  pd.DataFrame(clustering_coef.items(),columns=['cell_id','clustering_coef'])
        nodes = nodes.merge(clustering_df, on ='cell_id', how='left')
        nodes_stats['median_clustering_coefficient'] = np.median(clustering_df.clustering_coef.values)
        del clustering_df
        
        components = list(nx.connected_components(cell_graph))
        subgraphs = [cell_graph.subgraph(c).copy() for c in components]
        
        map_to_component = {}
        for index, component in enumerate(components) :
            elements_component = list(component)
            for element in elements_component:
                map_to_component[element] = index   
                
                
        nodes['component'] = nodes.reset_index()['cell_id'].apply(lambda x: map_to_component[x]if x in map_to_component.keys() else '') 
        edges['component'] = edges.cell_id_1.apply(lambda x: map_to_component[x] if x in map_to_component.keys() else '')
        
        
        size_df = nodes.groupby('component').agg('count').sort_values('cell_id')['cell_id'].reset_index().rename(columns = {'cell_id':'size'})
        nodes = nodes.merge(size_df,on='component',how='left')
        N = len(edges)
        r = int((N/1000)/2000)+1 
        q = [int(c) for c in np.linspace(0, N, r+2)]
        edges_merged = pd.DataFrame({})
        print('Dividing to '+ str(len(q)) + ' parts, done')
        for index in range(len(q))[1:]:
            lower_bound = q[index-1]
            upper_bound = q[index]
            edges_slice = edges[lower_bound:upper_bound].copy()
            edges_slice = edges_slice.merge(size_df,on='component',how='left')
            edges_merged = pd.concat([edges_merged,edges_slice])
    
        edges = edges_merged 
        del edges_merged
        
        print('Filtering components...')
        
        nodes = nodes[nodes['size']> MIN_ELEMENTS_PER_CLUSTER]
        edges = edges[edges['size']> MIN_ELEMENTS_PER_CLUSTER]
        print('DONE')

        print('Computing AUC per component...')

        AUC_per_component={}
        components_in_graph = edges.component.unique()
        for component in components_in_graph :
    
            edge_component_df = edges[edges.component == component].copy()
            distribution =  G_cross_function(edge_component_df,min_radius=10, max_radius=20,radius_numbers=20,TILs=['T','B'])
            area = compute_AUC(distribution)
            AUC_per_component[component] = area
    
        AUC_per_component = pd.melt(pd.DataFrame(AUC_per_component, index = [0])) 
        AUC_per_component = AUC_per_component.rename(columns = {'variable':'component','value':'AUC_area'})  
        
        edges = edges.merge(AUC_per_component,on = 'component', how ='left')
        nodes = nodes.merge(AUC_per_component,on = 'component', how ='left')
        
        print('DONE')
        
        
        fig = AUC_per_component.sort_values(['AUC_area']).plot.bar(x='component',y='AUC_area',rot=0,figsize =(40,10))
        plt.xlabel('Components')
        plt.ylabel('AUC area')
        plt.grid()
        plt.title('AUC area per component')
        plt.savefig(PLOT_PATH + 'AUC_area.png')
        plt.close()
        
        del AUC_per_component

        
        
        print('Count edges per component')
        edge_phenotype_count = edges.groupby(['component','phenotype_1','phenotype_2']).agg('count')[['cell_id_1']].reset_index().rename(columns = {'cell_id_1':'edge_count'})
        number_per_component = edges.groupby(['component']).agg('count')[['cell_id_1']].reset_index().rename(columns = {'cell_id_1':'total_edge_per_component'})
        edge_phenotype_count = edge_phenotype_count.merge(number_per_component,on='component',how='left')
        edge_phenotype_count['edge_count_ratio'] = edge_phenotype_count['edge_count']/edge_phenotype_count['total_edge_per_component']
        edge_phenotype_count['edge_type'] = edge_phenotype_count.apply(lambda row : order_phenotypes(row.phenotype_1,row.phenotype_2), axis=1)
        edge_phenotype_count = edge_phenotype_count[['component','edge_count_ratio','edge_type']]
        
        
        edge_phenotype_count_pivotted = pd.pivot_table(edge_phenotype_count, values='edge_count_ratio', index=['component'],
                    columns=['edge_type'], aggfunc=np.sum)

        edge_phenotype_count_pivotted.columns.name = None
        edge_phenotype_count_pivotted = edge_phenotype_count_pivotted.reset_index()
        edge_phenotype_count_pivotted = edge_phenotype_count_pivotted.fillna(0)
        print('DONE')
        
        columns_of_interest = edge_phenotype_count_pivotted.columns
        columns = edge_phenotype_count_pivotted.columns
        for interest in columns_of_interest:
            if not(interest in columns):
                edge_phenotype_count_pivotted[interest] = 0
                
        edges_of_interest = edge_phenotype_count_pivotted[columns_of_interest] 
        
        plot_relplot(edges_of_interest,'component','tumor-T','Reds','The of T-tumor edges across components',True)
        plt.close()

        plot_relplot(edges_of_interest,'component','tumor-B','Reds','The of B-tumor edges across components',True)
        plt.close()

        
        nodes = nodes.merge(edges_of_interest, on='component',how='left')
        del edges_of_interest
        del edge_phenotype_count_pivotted
        del edge_phenotype_count
        nodes = nodes.fillna(0)
        
        fig, ax = plt.subplots(figsize = (15,15))
        sns.scatterplot(data = nodes , x='cell_x_position', y= 'cell_y_position', hue = 'component').get_figure().savefig(PLOT_PATH+'scatter_component_250.png');
        plt.close()

        
        
        phenotype_component_grouped = pd.DataFrame(nodes.groupby(['component','phenotype']).agg('count')['cell_id']).reset_index()
        phenotype_component_grouped = phenotype_component_grouped.merge(size_df, on='component',how='left')
        phenotype_component_grouped['pheno_ratio'] = phenotype_component_grouped['cell_id']/phenotype_component_grouped['size']
        phenotype_component_pivotted = pd.pivot_table(phenotype_component_grouped, values='pheno_ratio', index=['component'],
                    columns=['phenotype'], aggfunc=np.sum)
        phenotype_component_pivotted.columns.name = None
        phenotype_component_pivotted = phenotype_component_pivotted.fillna(0).sort_values(['T']).reset_index()
        
        
        plot_relplot(phenotype_component_pivotted,'component','tumor',PLOT_PATH,'Reds','The number of tumorous cells per component',True)
        plt.close()

        plot_relplot(phenotype_component_pivotted,'component','T',PLOT_PATH,'Reds','The number of T cells per component',True)
        plt.close()

        plot_relplot(phenotype_component_pivotted,'component','B',PLOT_PATH,'Reds','The number of B cells per component',True)
        plt.close()

        
        phenotype_component_pivotted = phenotype_component_pivotted.merge(size_df, on='component',how='left')
        phenotype_component_pivotted = phenotype_component_pivotted[phenotype_component_pivotted.tumor != 0 ].copy()
        phenotype_component_pivotted = phenotype_component_pivotted.rename(columns = {'size_x':'size'})
        
        main_phenotypes = ['T','B','NK','stroma','tumor','dendtritic','macrophages']
        columns = phenotype_component_pivotted.columns
        for pheno in main_phenotypes:
            if not(pheno in columns):
                phenotype_component_pivotted[pheno] = 0
                
                
        phenotype_component_pivotted['T_tumor_density'] = phenotype_component_pivotted['T']/phenotype_component_pivotted['tumor']
        phenotype_component_pivotted['B_tumor_density'] = phenotype_component_pivotted['B']/phenotype_component_pivotted['tumor']
        phenotype_component_pivotted['NK_tumor_density'] = phenotype_component_pivotted['NK']/phenotype_component_pivotted['tumor']
        phenotype_component_pivotted['stroma_tumor_density'] = phenotype_component_pivotted['stroma']/phenotype_component_pivotted['tumor']
        phenotype_component_pivotted['T_density'] = phenotype_component_pivotted['T']/phenotype_component_pivotted['size']
        phenotype_component_pivotted['B_density'] = phenotype_component_pivotted['B']/phenotype_component_pivotted['size']
        phenotype_component_pivotted['NK_density'] = phenotype_component_pivotted['NK']/phenotype_component_pivotted['size']
        phenotype_component_pivotted['dendtritic_density'] = phenotype_component_pivotted['dendtritic']/phenotype_component_pivotted['size']
        phenotype_component_pivotted['macrophages_density'] = phenotype_component_pivotted['macrophages']/phenotype_component_pivotted['size']   
        
        
        plot_relplot(phenotype_component_pivotted,'component','T_tumor_density',PLOT_PATH,'Reds','T cells density over Tumor cells',True)
        plt.close()

        plot_relplot(phenotype_component_pivotted,'component','B_tumor_density',PLOT_PATH,'Reds','B cells density over Tumor cells',True)
        plt.close()

        plot_relplot(phenotype_component_pivotted,'component','NK_tumor_density',PLOT_PATH,'Reds','NK cells density over Tumor cells',True)
        plt.close()

        
        
        nodes = nodes.merge(phenotype_component_pivotted, on='component', how='left')
        
        del phenotype_component_pivotted
        del phenotype_component_grouped
        
        columns_to_avg = ['component','cell_x_position', 'cell_y_position', 'clustering_coef','missing_degree', 'stroma_degree', 'tumor_degree', 'total_degree',
       'B_degree', 'MISSING_degree', 'NK_degree', 'T_degree',
       'dendtritic_degree', 'macrophages_degree', 'tumor_pheno_degree', 'stroma_pheno_degree',
       'T','B', 'MISSING', 'NK', 'dendtritic', 'macrophages', 'stroma', 'tumor',
       'T_tumor_density', 'B_tumor_density', 'NK_tumor_density',
       'stroma_tumor_density', 'T_density', 'B_density', 'NK_density',
       'dendtritic_density', 'macrophages_density','AUC_area','B-B',
       'MISSING-B', 'MISSING-MISSING', 'T-B', 'T-MISSING', 'T-T',
       'dendtritic-B', 'dendtritic-MISSING', 'dendtritic-T',
       'dendtritic-dendtritic', 'macrophages-B', 'macrophages-MISSING',
       'macrophages-T', 'macrophages-dendtritic', 'macrophages-macrophages',
       'stroma-B', 'stroma-MISSING', 'stroma-T', 'stroma-dendtritic',
       'stroma-macrophages', 'stroma-stroma', 'tumor-B', 'tumor-MISSING',
       'tumor-T', 'tumor-dendtritic', 'tumor-macrophages', 'tumor-stroma',
       'tumor-tumor']

        all_columns = nodes.columns
        
        nodes_to_components = add_null_column(nodes,columns_to_avg)
        component_nodes = nodes_to_components.groupby(['component']).agg('mean').reset_index()
        #component_nodes = component_nodes_sum.merge(component_nodes_avg, on='component', how ='left')
        component_nodes = component_nodes.drop(['cell_id','on_border'],axis =1)
        
        component_nodes = nodes_to_components.groupby(['component']).agg('mean').reset_index()
        #component_nodes = component_nodes_sum.merge(component_nodes_avg, on='component', how ='left')
        component_nodes = component_nodes.drop(['cell_id','on_border'],axis =1)
        
        
        component_x = component_nodes.cell_x_position.values
        component_y = component_nodes.cell_y_position.values
        points = []
        for x,y in zip(component_x,component_y):
            points.append([x,y])
        points = np.array(points)    
        tri = Delaunay(points)
        
        
        fig = plt.figure(figsize=(15,7))
        plt.triplot(points[:,0], points[:,1], tri.simplices)
        plt.plot(points[:,0], points[:,1], 'o')
        plt.title('Delaunay component triangularisation')    
        plt.close()

        fig.savefig(PLOT_PATH+'Delaunay_component_triangularisation.png')
        
        threshold = 1000
        vertices = component_nodes.component.values
        coord, edges_component = get_high_level_graph(component_nodes,threshold)
        
        
        component_graph = nx.Graph() # for a directed graph use nx.DiGraph()
        component_graph.add_nodes_from(range(len(vertices)))  # add multiple nodes at once
        component_graph.add_edges_from(edges_component)
        
        
        laplacian_matrix = nx.laplacian_matrix(component_graph).A
        eigen_vals, eigen_vects = np.linalg.eig(laplacian_matrix)
        idx = (-eigen_vals).argsort()[::-1]   
        eigenValues = eigen_vals[idx]
        eigenVectors = eigen_vects[:,idx]
        
        SIGNAL = np.array(component_nodes.AUC_area.fillna(0).values)
        GRAPH = component_graph
        FIGNAME = 'AUC area signal projection on the eigenvector space'
        projection = project_signal(GRAPH,SIGNAL,PLOT_PATH,True,'b',True,FIGNAME)
        plt.close() 
        
        quadtratics = get_quadratic_laplacian_forms(component_nodes,vertices,edges_component)
        quadtratics = pd.melt(quadtratics)
        
        fig = quadtratics.sort_values(['value']).plot.bar(x='variable',y='value',rot=70,figsize =(20,10))
        plt.xlabel('Signals')
        plt.ylabel('Quadratic Laplace form')
        plt.grid()
        plt.title('Quadratic Laplace form for all signals')
        plt.savefig(PLOT_PATH + 'Quadratic Laplace form for all signals.png')
        plt.close()
        
        
        
        nodes.to_csv(OUTPUT_PATH + 'nodes_features.csv', index = False)
        component_nodes.to_csv(OUTPUT_PATH + 'component_features.csv', index = False)
        get_quadratic_laplacian_forms(component_nodes,vertices,edges_component).to_csv(OUTPUT_PATH + 'laplace_quadratic_form_high_level_graph.csv', index = False)
        nodes_stats.to_csv(OUTPUT_PATH + 'nodes_stats.csv', index = False)
        
        del nodes
        del edges
        del nodes_stats
        del component_nodes
                
        

Treating patient3P1
Importing edges...
DONE
Counting edges per phenotype...
DONE
Computing AUC...
DONE
Computing TILs...
DONE
Cell degree...
DONE
Building the whole graph...
DONE
Building the border graph...
Name: 
Type: Graph
Number of nodes: 1436398
Number of edges: 6090851
Average degree:   8.4807
Graph is not connected
Diameter and Avg shortest path length are not defined!
Sparsity: 0.0000
Global clustering coefficient aka Transitivity: 0.5199
DONE
Connected components...
The cell graph contains 1138 connected components
The largest component has 1315814 nodes accounting for 91.61 % of the nodes
231  components have at least  100  nodes
